In [31]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [32]:
import sys
sys.path.append("")

In [33]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [34]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments


# from vncorenlp import VnCoreNLP
import torch 
import nltk
from nltk.tokenize import sent_tokenize



In [35]:
OUTPUT_DIR = 'vietnamese_t5_summary_model_falcon'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [36]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Processing data

In [37]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [38]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [39]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [40]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [41]:
train_df = get_dataframe(train_paths[:100])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [42]:
val_df = get_dataframe(val_paths[:100])
test_df = get_dataframe(test_paths[:100])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [43]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/00444...,Chiến_dịch do Công_an tỉnh Sơn_La phối_hợp các...,Hai tay trùm đang trốn truy_nã điều_hành đường...
1,data/vietnews-master/data/test_tokenized/00627...,"Trước tình_trạng ngập_lụt , các quốc_gia tiên_...","Những quốc_gia tiên_tiến như Nhật_Bản , Hà_Lan..."
2,data/vietnews-master/data/test_tokenized/01640...,"Sau 4 ngày nghị_án , sáng nay 19-6 Toà_án nhân...",Hội_đồng xét_xử xét thấy sự_cố chạy thận là hi...
3,data/vietnews-master/data/test_tokenized/00352...,Dự_án cải_tạo Bến xe_buýt Tân_Quy với kinh_phí...,Trung_tâm Quản_lý giao_thông công_cộng TP. HCM...
4,data/vietnews-master/data/test_tokenized/00682...,"Tờ al - Watan hôm 14/3 cho_hay , các máy_bay c...",Các đơn_vị không_quân Syria và Nga đã tiến_hàn...


In [44]:
# test_df.to_parquet("data/vietnews/test.parquet", index=False)

In [45]:
# train_df = pd.read_parquet("data/vietnews/train.parquet")
# val_df = pd.read_parquet("data/vietnews/val.parquet")
# test_df = pd.read_parquet("data/vietnews/test.parquet")
# test_df.head()

In [46]:
# import matplotlib.pyplot as plt

# train_df['word_count'] = train_df['summary'].apply(lambda x: len(x.split()))

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(train_df['word_count'], bins=100, edgecolor='black')
# plt.title('Histogram of Number of Words in Sentences')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [47]:
# pretrained_model = "google/mt5-small"
pretrained_model = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained("Vietnam_T5_small_200")


OSError: Vietnam_T5_small_200 does not appear to have a file named config.json. Checkout 'https://huggingface.co/Vietnam_T5_small_200/tree/None' for available files.

In [ ]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [ ]:
batch_size=8  # change to 16 for full training

max_input_length = 512 #1024
max_target_length = 128 #128


def process_data_to_model_inputs(batch):
    model_inputs = tokenizer(
        batch["original"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        batch["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map: 100%|██████████| 5000/5000 [00:04<00:00, 1179.01 examples/s]


In [ ]:
tokenizer.pad_token_id

0

In [ ]:
train_df.iloc[0]

file        data/vietnews-master/data/train_tokenized/0848...
original    Dự_án nào đang vướng thủ_tục pháp_lý , dự_án n...
summary     Những ngày qua , thông_tin hơn 100 dự_án bất_đ...
Name: 0, dtype: object

In [ ]:
a = tokenizer(str(train_df["summary"].iloc[0]))
print(str(train_df["summary"].iloc[0]))
a

Những ngày qua , thông_tin hơn 100 dự_án bất_động_sản ở TP. HCM bị " đóng_băng " , rồi lại đến tin 124 dự_án được tháo_gỡ , có_khi là 160 dự_án vướng_mắc … khiến những người đã và đang có ý_định mua căn_hộ càng lo_lắng . 


{'input_ids': [445, 107, 2, 1725, 3, 1725, 85, 63, 3, 4960, 3, 6, 3, 189, 10079, 1725, 834, 17, 77, 3, 107, 2, 29, 910, 3, 26, 2, 834, 12916, 3, 115, 2, 17, 834, 2, 1725, 834, 7, 2, 29, 3, 2, 3, 7150, 5, 454, 5518, 3, 115, 2, 96, 3, 2, 15742, 122, 834, 115, 98, 1725, 96, 3, 6, 3, 52, 2, 23, 3, 40, 2, 23, 3, 2, 29, 3, 17, 77, 3, 22504, 3, 26, 2, 834, 12916, 3, 2, 75, 3, 189, 2975, 32, 834, 122, 2, 3, 6, 3, 75, 4922, 834, 29392, 5381, 11321, 3, 26, 2, 834, 12916, 3, 208, 2, 1725, 834, 51, 2, 75, 3, 233, 3, 29392, 2, 29, 3, 29, 107, 2, 1725, 3, 1725, 2, 23, 3, 2, 3, 208, 85, 3, 2, 1468, 3, 75, 4922, 3, 2, 834, 2, 29, 107, 4035, 9, 375, 29, 834, 107, 2, 3, 75, 85, 1725, 6899, 834, 40, 2, 1725, 3, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
tokenizer.convert_ids_to_tokens(train_data_batch[0]["labels"])[:10]

['▁N', 'h', '<unk>', 'ng', '▁', 'ng', 'à', 'y', '▁', 'qua']

In [24]:
# train_data_batch[0]

### **Warm-starting the Encoder-Decoder Model**

In [25]:
from transformers import AutoModelForSeq2SeqLM, GenerationConfig

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model).to(device)
my_config = model.generation_config
print("generation_config", model.generation_config)

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}



### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [26]:
import numpy as np
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt')

In [28]:
features = [train_data_batch[i] for i in range(2)]
data_collator(features)

/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'input_ids': tensor([[ 309,    2,  834,  ..., 1725,  834,    1],
        [ 445,  122,   85,  ..., 2975,  102,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  445,   107,     2,  1725,     3,  1725,    85,    63,     3,  4960,
             3,     6,     3,   189, 10079,  1725,   834,    17,    77,     3,
           107,     2,    29,   910,     3,    26,     2,   834, 12916,     3,
           115,     2,    17,   834,     2,  1725,   834,     7,     2,    29,
             3,     2,     3,  7150,     5,   454,  5518,     3,   115,     2,
            96,     3,     2, 15742,   122,   834,   115,    98,  1725,    96,
             3,     6,     3,    52,     2,    23,     3,    40,     2,    23,
             3,     2,    29,     3,    17,    77,     3, 22504,     3,    26,
             2,   834, 12916,     3,     2,    75,     3,   189,  2975,    32,
           834,   122,     2,     3,     6,     3,    75,  4922,   834,

Cool! Finally, we start training.

In [29]:
# set training arguments - these params are not really tuned, feel free to change
logging_steps = min(2000,len(train_data_batch) // batch_size)
print("logging_steps", logging_steps)

training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    # do_train=True,
    # do_eval=True,
    logging_steps=logging_steps,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=750,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    optim='adafactor',
    save_total_limit=3,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

logging_steps 2000


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.539300,1.422631,15.957100,5.680300,13.707100,13.812700
2,1.425500,1.342705,16.303900,6.075700,14.003100,14.111400
3,1.374700,1.292822,16.285200,6.139000,14.011900,14.120900
4,1.334700,1.262241,16.448100,6.271400,14.170600,14.280600
5,1.318600,1.239643,16.521300,6.430700,14.289000,14.385300
6,1.297300,1.224418,16.424400,6.384000,14.216700,14.318800
7,1.281700,1.213849,16.572700,6.468500,14.355800,14.470300


/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/g

KeyboardInterrupt: 

In [30]:
trainer.save_model(OUTPUT_DIR)

model.safetensors: 100%|██████████| 242M/242M [00:38<00:00, 6.26MB/s] 


In [ ]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained(OUTPUT_DIR, torch_dtype=torch.bfloat16,).to(device)
print("generation_config", model.generation_config)


# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask,
                             max_length=128,
                        # num_beams=4,
                        # no_repeat_ngram_size=3,
                        # early_stopping=True,
                        )

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Parameter 'function'=<function generate_summary at 0x7f954f5cecb0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous 

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}



Map:   0%|          | 0/100 [00:00<?, ? examples/s]/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:01<00:00, 50.35 examples/s]


In [ ]:
rouge_output = rouge_score.compute(predictions=pred_str, references=label_str, use_stemmer=True)
rouge_output = {key: value * 100 for key, value in rouge_output.items()}
rouge_output

{'rouge1': 11.615472530587926,
 'rouge2': 2.5109031615037156,
 'rougeL': 9.59317086900173,
 'rougeLsum': 9.598338898153589}

In [ ]:
for i in range(10):
    print('\nPrediction: ',pred_str[i])
    print('Truth: ',label_str[i])
    print('Content: ',test_data[i]['original'])
    


Prediction:  Các nhà_t_chc cho_hay
Truth:  Hàng trăm_nghìn người Hàn_Quốc hôm_nay xuống_đường ở Seoul trong cuối tuần thứ tư liên_tiếp để phản_đối Tổng_thống Park_Geun-hye sau bê_bối liên_quan tới bạn thân . 
Content:  Các nhà_tổ_chức cho_hay có 500.000 người đã tham_gia biểu_tình tối nay . Cuộc biểu_tình tối nay có quy_mô nhỏ hơn những lần trước khi các hoạt_động tương_tự cũng diễn ra ở khắp nhiều thành_phố khác . Cảnh_sát cho_hay ít_nhất 155.000 người đã tập_trung tại một quảng_trường trung_tâm Seoul vào đầu buổi tối để thắp nến , trong khi các nhà_tổ_chức nói rằng con_số này lên tới 500.000 người . Tổng_thống Park từ_chối các lời kêu_gọi từ_chức , bất_chấp cuộc khủng_hoảng_chính_trị đang diễn ra , trong đó bà bị_cáo buộc để Choi_Soon - sil , một người bạn thân , can_thiệp vào công_việc nhà_nước và trục_lợi cho cá_nhân . Bà thừa_nhận sai_sót trên và cam_kết hợp_tác với các nhà điều_tra vụ bê_bối . Các công_tố_viên dự_kiến sẽ trình cáo_trạng chống lại bà Choi và hai cựu cố_vấn tổng_t

In [ ]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/010067.txt.seg'